# Intel Image Classification using Pytorch

## Load Dependencies

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
from torch.utils.data import random_split

## Importing Data (Train, validation, test)

In [133]:
transform = tt.Compose([
    tt.Resize(64),
    tt.RandomCrop(64),
    tt.ToTensor()
])

BATCH_SIZE = 128
train = ImageFolder(root = "C:/Users/Ineed/OneDrive/Bureau/GITHUB/Intel Image Classification/seg_train", transform = transform)
train_dl = DataLoader(train, batch_size = BATCH_SIZE, shuffle=True, num_workers=3, pin_memory=True)

In [134]:
mean, std, nb_batch = 0.0, 0.0, 0
for data, label in train_dl :
    mean += torch.mean(data, dim = [0,2,3])
    std += torch.mean(data*data, dim = [0,2,3])
    nb_batch += 1
    
mean /= nb_batch
std = (std/nb_batch - mean**2)**0.5

In [135]:
mean, std

(tensor([0.4302, 0.4575, 0.4538]), tensor([0.2483, 0.2467, 0.2806]))

In [136]:
norm = (tuple(mean.numpy()), tuple(std.numpy()))

In [137]:
train_transform = tt.Compose([
    tt.Resize(64),
    tt.RandomCrop(64),
    tt.RandomHorizontalFlip(p = 0.5),
    tt.ToTensor(),
    tt.Normalize(mean = norm[0], std = norm[1], inplace = True)
])


test_transform = tt.Compose([
    tt.Resize(64),
    tt.RandomCrop(64),
    tt.ToTensor(),
    tt.Normalize(mean = norm[0], std = norm[1], inplace = True)
])

In [138]:
train = ImageFolder(root = "C:/Users/Ineed/OneDrive/Bureau/GITHUB/Intel Image Classification/seg_train", transform = train_transform)
test = ImageFolder(root = "C:/Users/Ineed/OneDrive/Bureau/GITHUB/Intel Image Classification/seg_test", transform = test_transform)

In [139]:
num_items = len(train)
num_train = round(num_items * 0.8)
num_val = num_items - num_train

train_ds, val_ds = random_split(train, [num_train, num_val])

In [140]:
BATCH_SIZE = 128
train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=3, pin_memory=True)
valid_dl = DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=3, pin_memory=True)
test_dl = DataLoader(test, batch_size = BATCH_SIZE, shuffle=False, num_workers=3, pin_memory=True)

In [167]:
class CNN(nn.Module) :
    
    def __init__(self) :
        super().__init__()
    
        self.Convolve1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
        )
        self.Convolve2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
        )
        
        self.Convolve3 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(512),
            nn.SiLU(),
            nn.Conv2d(512, 1024, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(1024),
            nn.SiLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024*8*8, 512),
            nn.SiLU(),
            nn.Linear(512, 64),
            nn.SiLU(),
            nn.Linear(64, 6)
        )

    
    def forward(self, x) :
        x = self.Convolve1(x)
        x = self.Convolve2(x)
        x = self.Convolve3(x)
        x = self.fc(x)
        return x

In [168]:
def inference (model, val_dl):
    correct_prediction = 0
    total_prediction = 0

    # Disable gradient updates
    with torch.no_grad():
            for data, label in val_dl:
                # Get the input features and target labels, and put them on the GPU
                inputs, labels = data.to(device), label.to(device)

                # Normalize the inputs
                inputs_m, inputs_s = inputs.mean(), inputs.std()
                inputs = (inputs - inputs_m) / inputs_s

                # Get predictions
                outputs = model(inputs)

                # Get the predicted class with the highest score
                _, prediction = torch.max(outputs,1)
                # Count of predictions that matched the target label
                correct_prediction += (prediction == labels).sum().item()
                total_prediction += prediction.shape[0]

    acc = correct_prediction/total_prediction
    print(f'Test Accuracy: {acc:.2f}, Total items: {total_prediction}')

    return acc

In [169]:
model = CNN()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = model.to(device)
next(model.parameters()).device


num_epochs = 5

In [170]:


acc_train = []
acc_test = []
losses = []

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0
    i = 0

    for data, label in train_dl:
        print("Batch {}".format(i))
        i+=1
        inputs, labels = data.to(device), label.to(device)

        
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s
        
        # Zero the parameter gradients
        optimizer.zero_grad(set_to_none=True)

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs,1)
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

    nb_batches = len(train_dl)
    avg_loss = running_loss / nb_batches
    acc = correct_prediction/total_prediction
    
    acc_train.append(acc)
    losses.append(avg_loss)
    
    
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Train Accuracy: {acc:.2f}')
    acc_test.append(inference(model, valid_dl))

print('Finished Training')

Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Epoch: 0, Loss: 0.88, Train Accuracy: 0.66
Test Accuracy: 0.75, Total items: 2807
Batch 0
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Ba

In [6]:
losses = [0.88, 0.58, 0.48, 0.43, 0.37]
train_acc = [0.66, 0.79, 0.83, 0.84, 0.87]
test_acc = [0.75, 0.78, 0.79, 0.82, 0.83]
x = np.arange(len(losses))

In [ ]:
plt.plot(x, losses)